<a href="https://colab.research.google.com/github/khalilrbh/PFE/blob/main/PFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import shutil

# Set dataset path
DATASET_PATH = "C:/Users/Lenovo/Desktop/PFE/state-farm-distracted-driver-detection"

In [17]:
# Define image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 10  # c0 to c9

# Map class labels
CLASS_LABELS = {
    'c0': 'Safe driving',
    'c1': 'Texting - right',
    'c2': 'Talking on the phone - right',
    'c3': 'Texting - left',
    'c4': 'Talking on the phone - left',
    'c5': 'Operating the radio',
    'c6': 'Drinking',
    'c7': 'Reaching behind',
    'c8': 'Hair and makeup',
    'c9': 'Talking to passenger'
}

# Create dataset structure
def create_data_folders():
    train_dir = os.path.join(DATASET_PATH, "train")
    new_train_dir = os.path.join(DATASET_PATH, "train_images")
    os.makedirs(new_train_dir, exist_ok=True)

    df = pd.read_csv(os.path.join(DATASET_PATH, "driver_imgs_list.csv"))

    for c in CLASS_LABELS.keys():
        os.makedirs(os.path.join(new_train_dir, c), exist_ok=True)

    for _, row in df.iterrows():
        class_folder = os.path.join(new_train_dir, row['classname'])
        shutil.move(os.path.join(train_dir, row['img']), os.path.join(class_folder, row['img']))

create_data_folders()

# Image Data Generators
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    os.path.join(DATASET_PATH, "train_images"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator = datagen.flow_from_directory(
    os.path.join(DATASET_PATH, "train_images"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# Define CNN Model
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation="relu", input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation="relu"),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation="softmax")
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model

model = build_model()

# Train Model
EPOCHS = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

# Save Model
model.save("distracted_driver_model.h5")

# Plot Accuracy & Loss
def plot_results(history):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title("Accuracy")

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title("Loss")

    plt.show()

plot_results(history)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Lenovo/Desktop/PFE/state-farm-distracted-driver-detection/driver_imgs_list.csv'